In [104]:
import json
import requests
import gradio as gr
from sentence_transformers import SentenceTransformer
from embed_chromadb_with_confluence import EmbedChromaDBWithConfluenceContent
# Query the results from the DB
model = SentenceTransformer("all-MiniLM-L6-v2")
def chat(message,history):
    q_embedding = model.encode([message])[0]
    chromadb = EmbedChromaDBWithConfluenceContent()
    results = chromadb.get_collection().query(query_embeddings=[q_embedding], n_results=3)
    context = "\n\n".join(results["documents"][0])
    print(context)
    prompt = f"""You are a helpful assistant. 
        Answer the question using only the context below:
        Context:
        {context}
        Question: {message}
        Answer:"""
    print(prompt)
    stream = requests.post('http://localhost:11434/api/generate', json={"model": "llama3.2", "prompt": prompt, "stream": True})
    response = ""
    for line in stream.iter_lines():
        if line:
            try:
                data = json.loads(line.decode("utf-8"))
                if "response" in data:
                   response+=data["response"] or ''
                   yield response
                if data.get("done", False):
                    print("\n--- Generation Complete ---")
                    break
            except json.JSONDecodeError:
                pass

In [ ]:
gr.ChatInterface(fn=chat,type="messages", title="Confluence RAG with LLM", 
                 description="Ask questions about your Confluence content.").launch()

* Running on local URL:  http://127.0.0.1:7894
* To create a public link, set `share=True` in `launch()`.


Link</u></a></p></li></ul><hr /><h2>📫 Connect with Me</h2><ul><li><p>📧 Email: <a href="mailto:vinaykumarreddy.chaitu@outlook.com"><strong><u>vinaykumarreddy.chaitu@outlook.com</u></strong></a></p></li><li><p>💼 LinkedIn: <a href="https://www.linkedin.com/in/vinaykumarreddy-satti-4883a91ab"><u>linkedin.com/in/vinaykumarreddy-satti-4883a91ab</u></a></p></li></ul><hr /><p>⭐️ From <a href="https://github.com/vinaykumarreddysatti/my-profile/tree/main"><u>Vinay Kumar Reddy</u></a></p>

<h1>👋 Hi, I'm Vinay Kumar Reddy Satti</h1><p>🚀 Full-stack developer with <strong>9.5+ years of experience</strong> in building scalable, high-performance applications across web, mobile, and backend systems. Passionate about clean architecture, optimized workflows, and delivering impactful solutions.</p><hr /><h2>🧑&zwj;💻 About Me</h2><ul><li><p>🔭 Currently working as a <strong>Lead Developer at TCS (Bangkok)</strong></p></li><li><p>🌱 Pursuing a <strong>Master&rsquo;s in AI &amp; ML</strong> while working full-t